In [3]:
!wget --no-check-certificate \
   https://github.com/ashlizjosh/Miniproject/raw/master/rice-leaf.zip
   #https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  #-O cats_and_dogs_filtered.zip
  #rice_diseases.zip

--2021-01-23 06:44:20--  https://github.com/ashlizjosh/Miniproject/raw/master/rice-leaf.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ashlizjosh/Miniproject/master/rice-leaf.zip [following]
--2021-01-23 06:44:21--  https://raw.githubusercontent.com/ashlizjosh/Miniproject/master/rice-leaf.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7816855 (7.5M) [application/zip]
Saving to: ‘rice-leaf.zip’

rice-leaf.zip       100%[===================>]   7.45M  24.9MB/s    in 0.3s    

2021-01-23 06:44:22 (24.9 MB/s) - ‘rice-leaf.zip’ saved [7816855/7816855]



In [5]:
! unzip rice-leaf.zip

Archive:  rice-leaf.zip
  inflating: rice/validation/brownspot/brownspot_rotated_009.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_017.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_007.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_014.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_008.png  
  inflating: rice/validation/brownspot/brownspot_rotated_001.jpg  
  inflating: rice/train/blast/blast_rotated_042.png  
  inflating: rice/validation/brownspot/brownspot_rotated_005.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_003.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_013.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_011.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_010.jpg  
  inflating: rice/validation/brownspot/brownspot_rotated_016.jpg  
  inflating: rice/train/blast/blast_rotated_046.png  
  inflating: rice/validation/brownspot/brownspot_rotated_012.jpg

In [6]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [8]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/rice/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/rice/validation", target_size=(224,224))

Found 157 images belonging to 3 classes.
Found 84 images belonging to 3 classes.


In [26]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    return model
    

In [28]:
model=VGG16()

In [29]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 56, 56, 256)      

In [41]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [42]:
hist = model.fit_generator(traindata,validation_data=testdata,epochs=10,verbose=1,callbacks=[checkpoint, early])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10


InvalidArgumentError: ignored

In [19]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

NameError: ignored